In [1]:
from english_words import english_words_set
import pandas as pd
import numpy as np

In [7]:
# make a list of 5 lenght words without non-alphas, and no proper nouns
words5 = []

for word in english_words_set:
    if len(word) == 5 and word[0].islower() and word.isalpha():
        words5.append(word)

In [8]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [18]:
df_words = pd.DataFrame(words5)
df_words.columns=['words']

In [26]:
df_words['first'] = df_words['words'].str.slice(0,1)

In [27]:
df_words

,words,first
0,llama,l
1,chaos,c
2,monad,m
3,madam,m
4,annoy,a
...,...,...
2465,swank,s
2466,opium,o
2467,utile,u
2468,cream,c
